In [12]:
import collections
import itertools

traDatas = ['abe','ae','abc','ade']

class Apriori:
    traDatas = []  # 临时的（当前的）数据集合
    traLen = 0  # 当前的数据集合的长度
    k = 1  # 频繁k项集，从1开始
    traCount = {}  # 对当前的数据集合的各个元素进行计数的集合
    freTran = {}  # 对频繁元素进行储存
    sup = 0  # 支持度
    conf = 0  # 置信度
    freAllTran = {}  # 所有的频繁转移集
    
    def __init__(self,traDatas,sup,conf):
        self.traDatas = traDatas  # 数据集合
        self.traLen = len(traDatas)  # 数据集合的长度
        self.sup = sup  # 支持度
        self.conf = conf  # 置信度
        
    def scanFirDatas(self):
        """计算每个元素的频率"""
        tmpStr = ''.join(traDatas)  # 将所有数据合并
        self.traCount = dict(collections.Counter(tmpStr))  # 对每个元素进行计数
        return self.traCount

    def getFreSet(self):
        self.freTran = {}
        for tra in self.traCount.keys():  # 遍历所有的元素
            if self.traCount[tra] >= self.sup and len(tra) == self.k: # 对于长度为k的元素，选取其数量大于sup的元素作为频繁集
                self.freTran[tra] = self.traCount[tra]
                self.freAllTran[tra] = self.traCount[tra]
                
    def cmpTwoSet(self,setA,setB):
        """若两个集合中有n-1个元素相同，则返回True"""
        setA = set(setA)
        setB = set(setB)
        if len(setA - setB) == 1 and len(setB - setA) == 1:
            return True
        else:
            return False
                
    def selfConn(self):
        self.traCount = {}  # 清空traCount
        for item in itertools.combinations(self.freTran.keys(),2): # 对于上一次得到的频繁集中的元素进行两两组合
            if self.cmpTwoSet(item[0], item[1]) == True :  # 若两个集合中有n-1个元素相同
                key = ''.join(sorted(''.join(set(item[0]).union(set(item[1])))))
                # 上一句的含义为：
                # 先对两个集合求并集
                # 再用join函数将其合并成string
                # 再对其按字母表进行排序，转换成list（我认为不需要，因为求并集的时候已经是从前之后进行排序了）
                # 再将list重新转换为string（同样不需要）
                # 总之，就是先取并集再转换成string
                if self.cutBranch(key) != False:  
                    # 对于得到的string的各个k元子集，若这些子集存在非频繁集，
                    # 则将该string对应的集合的计数值设为0，即等价于将其判断为非频繁集
                    self.traCount[key] = 0
                    
    def scanDatas(self):
        self.k = self.k + 1  # k自增
        for tra in traDatas:
            for key in self.traCount.keys():
                self.traCount[key] = self.traCount[key] + self.findChars(tra, key)
                    
                    
    def cutBranch(self, key):
        """剪枝，对于得到的string的各个k元子集，若这些k元子集存在非频繁集，则返回False"""
        for subKey in list(itertools.combinations(key,self.k)):
            if ''.join(list(subKey)) not in self.freTran.keys():
                return False
            
    def findChars(self, str_, chars):
        """查看某一字符串中是否有某个字符"""
        for char in list(chars):
            if char not in str_:
                return False
        return 1
    
    def permutation(self, string, pre_str, container):
        if len(string) == 1:
            container.append(pre_str + string)
        for idx, str_ in enumerate(string):
            new_str = string[:idx] + string[idx + 1:]
            new_pre_str = pre_str + str_
            self.permutation(new_str, new_pre_str, container)
            
    def genAssRule(self):
        container = []
        ruleSet = set()
        for item in self.freTran.keys():
            self.permutation(item, '', container)
        for item in container:
            for i in range(1, len(item)):
                print item[:i] + "" + item[i:]
                ruleSet.add((''.join(sorted(item[:i])), ''.join(sorted(item[i:]))))
        for rule in ruleSet:
            if self.calcConfi(rule[0],rule[1]) > self.conf:  # 置信度
                print rule[0] + '------>>>' + rule[1]
                
    def calcConfi(self,first,last):
        return self.freAllTran[''.join(sorted(first+last))]/self.freAllTran[''.join(sorted(first))]
    
    
    def algorithm(self):
        self.scanFirDatas()   # 计算了每个元素的频数
        while self.traCount != {}:  # 直到计数集合为空，否则一直循环
            self.getFreSet()
            self.selfConn()
            self.scanDatas()
        print self.freAllTran
        print self.freTran
        self.genAssRule()
        
apriori = Apriori(traDatas, 2, 0.7)
apriori.algorithm()

{'a': 4, 'ae': 3, 'ab': 2, 'b': 2, 'e': 3}
{'ab': 2, 'ae': 3}
ab
ba
ae
ea
b------>>>a
e------>>>a


In [18]:
traDatas = ['acde','abcdr','cde','acd','ac','df']
apriori = Apriori(traDatas, 2, 0.7)
apriori.algorithm()

{'a': 4, 'cde': 2, 'c': 5, 'ab': 2, 'ae': 3, 'd': 5, 'acd': 3, 'de': 2, 'ac': 4, 'ce': 2, 'cd': 4, 'b': 2, 'e': 2, 'ad': 3}
{'cde': 2, 'acd': 3}
cde
cde
ced
ced
dce
dce
dec
dec
ecd
ecd
edc
edc
acd
acd
adc
adc
cad
cad
cda
cda
dac
dac
dca
dca
ce------>>>d
de------>>>c
ad------>>>c
e------>>>cd
